# Vítej!

Tento dokument obsahuje kód k vytváření základní animace.

In [1]:
from manim import *

Manim Community v0.17.3

## Vlastní animace

In [2]:
%%manim -v WARNING -qh Intro

        
class Intro(Scene):
    def construct(self):
        square = Square().shift(LEFT * 1.5)
        circle = Circle().shift(RIGHT * 1.5)

        self.play(Create(square), Create(circle))
        
        self.play(FadeOut(square), FadeOut(circle))